In [1]:
import numpy as np
import nnfs
nnfs.init()
from nnfs.datasets import spiral_data

In [2]:
# Dense layer
class Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

# ReLU activation
class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

# Softmax activation
class Activation_Softmax:
    def forward(self, inputs):
        # Get unnormalized probabilities
        # Substracting from inputs as exp will be very high, linear modification doesn't effect the output
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        
        self.output = probabilities

# Common loss class
class Loss:
    # Calculates the data and regularization losses given model output and ground truth values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)

        # Calculate mean loss
        data_loss = np.mean(sample_losses)

        return data_loss
        
# Cross-entropy loss
class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = len(y_pred)

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values - only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

In [44]:
# Dataset
X, y = spiral_data(samples=100, classes = 3)

# Create a Dense layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

# Create ReLU activation (to be used with Dense layer)
activation1 = Activation_ReLU()

# Create second Dense Layer with 3 input features (as we take output of previous layer here) and 3 output values
dense2 = Layer_Dense(3,3)

# Create softmax activation (to be used with Dense layer)
activation2 = Activation_Softmax()

# Create loss function
loss_function = Loss_CategoricalCrossentropy()

# Make forward pass of our training data through this layer
dense1.forward(X)

# Make forward pass through activation function it takes the output of first dense layer
activation1.forward(dense1.output)

# Make a forward pass through second Dense layer it takes outputs from activation function of first layer as inputs
dense2.forward(activation1.output)

# Make a forward pass throught activation function it takes the output of second dense layer here
activation2.forward(dense2.output)

# out of first 5 samples
print(activation2.output[:5])

# Perform a forward pass through activation function it takes the output of second dense layer here and returns loss
loss = loss_function.calculate(activation2.output, y)

print('loss:', loss)

# Calculate accuracy from output of activation2 and targets
# Calculate values along first axis
predictions = np.argmax(activation2.output, axis=1)
# print(len(y.shape), 'len', predictions, y, predictions == y)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

print('acc:', accuracy)

[[0.33333334 0.33333334 0.33333334]
 [0.33333302 0.33333313 0.33333385]
 [0.33333257 0.33333287 0.3333346 ]
 [0.33333215 0.33333263 0.33333522]
 [0.33333185 0.33333242 0.33333576]]
loss: 1.0986161
acc: 0.31333333333333335
